In [15]:
import json
import requests
import ujson as json
from openai import OpenAI

import os

os.environ["OPENAI_API_KEY"] = "3bc24a63-eb36-43b3-ac21-c944005cf008"
# url无需添加具体接口的后缀，openai的sdk会自动补全
# 若是通过http方式调用则需要完整的接口url
os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"


def open_ai_sdk():
    client = OpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
        base_url=os.environ["OPENAI_API_BASE"],
    )
    # 此处传入headers中的Authorization 与在client传api_key是一样的
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
    }
    messages = [
        {"role": "system", "content": "你是一个聊天机器人"},
        {"role": "user", "content": "你是谁？"}
    ]
    # 本示例为请求聊天完成接口，如果需要请求别的接口请修改
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",  # 模型名称
        messages=messages,         # 聊天上下文
        temperature=0.5,           # 模型响应的温度参数
        max_tokens=50,             # 最大响应token数
        top_p=1,                   # top-p参数
        frequency_penalty=0,       # 频率惩罚
        presence_penalty=0,        # 存在惩罚
        stream=False               # 是否流式返回结果
        # 入参时erp改为不必填 但如果输入了erp会校验erp是否真实存在，erp仅用于观测实际调用人
        # extra_body={"erp": "python"}
        # 请求头
        # extra_headers=headers

    )

    return response


if __name__ == '__main__':
    print(open_ai_sdk())


ChatCompletion(id='chatcmpl-9jhhtqy0ZreywviRZH8WdFAshIyvY', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='你好！我是一个人工智能聊天机器人，可以帮助你回答问题、提供信息、进行对话和娱乐等。如果你有任何问题', role='assistant', function_call=None, tool_calls=None))], created=1720679797, model='gpt-4', object='chat.completion', service_tier=None, system_fingerprint='fp_811936bd4f', usage=CompletionUsage(completion_tokens=50, prompt_tokens=25, total_tokens=75))


In [50]:
import json
import os
from openai import OpenAI
import logging

# 设置日志记录
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 从环境变量中获取API密钥和基础URL
os.environ["OPENAI_API_KEY"] = "3bc24a63-eb36-43b3-ac21-c944005cf008"
os.environ["OPENAI_API_BASE"] = "http://gpt-proxy.jd.com/gateway/azure"

def open_ai_sdk(model_name, messages, temperature=0.5, max_tokens=1000):
    try:
        client = OpenAI(
            api_key=os.environ["OPENAI_API_KEY"],
            base_url=os.environ["OPENAI_API_BASE"],
        )

        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stream=False
        )
        logger.info("API response received.")
        return response
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return None


In [35]:
if __name__ == '__main__':
    new_messages = [
        {"role": "system", "content": "你是一个帮助者。"},
        {"role": "user", "content": "我想问一下LDA是什么？"},
        # 添加一个中文提示
        {"role": "assistant", "content": "LDA（潜在狄利克雷分配）是一种统计模型，用于发现文档集合中的主题。"}
    ]
    response = open_ai_sdk("gpt-4-1106-preview", new_messages)
    if response:
        try:
            # 假设 ChatCompletion 对象有一个 .data 属性或 .to_dict() 方法来获取响应数据
            # 根据实际情况修改以下代码
            if hasattr(response, 'data'):
                response_data = response.data
            elif hasattr(response, 'to_dict'):
                response_data = response.to_dict()
            else:
                # 如果这两个属性/方法都不存在，你可能需要根据实际情况来调整
                raise AttributeError("ChatCompletion 对象没有可访问的数据属性或方法")

            # 输出聊天机器人的回复内容
            assistant_reply = response_data["choices"][0]["message"]["content"]
            print("聊天机器人的回复：", assistant_reply)
        except AttributeError as e:
            print("访问响应数据时出错：", e)
        except KeyError as e:
            print("响应数据中的键错误：", e)
    else:
        print("未能从 OpenAI API 获取有效响应。")


INFO:httpx:HTTP Request: POST http://gpt-proxy.jd.com/gateway/azure/chat/completions "HTTP/1.1 200 "
INFO:__main__:API response received.


聊天机器人的回复： LDA全称为“Latent Dirichlet Allocation”，是一种常用于文本数据的主题模型。它能够从文档集合中自动地发现主题，并将文档表示为这些主题的混合。LDA背后的基本思想是，文档是由隐含的主题分布生成的，而每个主题又是由一系列单词的分布构成的。

具体来说，LDA模型假设：

1. **每个文档表示为隐含主题的混合**。每个文档都可以看作是多个主题的集合，但每个主题在不同文档中的比例不尽相同。
2. **每个主题表示为单词的分布**。每个主题都可以通过它包含的单词及其概率来描述，不同主题包含不同的单词集合。

LDA的工作流程大致可以分为以下几个步骤：

1. **确定主题数量**：在运行LDA模型之前，需要预先设定要提取的主题数量。这个数量可以根据实际需要和数据集的大小来确定。
2. **随机分配**：一开始，LDA会随机给文档中的每个单词分配一个主题。
3. **迭代更新**：通过迭代过程，LDA逐渐调整单词与主题之间的分配，以便最大化模型的联合概率。这个过程通常使用吉布斯采样（Gibbs sampling）或变分贝叶斯推断（Variational Bayes inference）等技术。
4. **得到结果**：经过足够的迭代后，每个文档的主题分布和每个主题的单词分布逐渐稳定。这时，可以根据这些分布来分析文档的主题结构。

LDA模型在文本挖掘、自然语言处理等领域有广泛的应用，如文档分类、信息检索、情感分析等。通过LDA，我们可以更好地理解和解释大规模文档集合的内容和结构。


## 模拟rag

In [51]:
import logging

# 假设的检索系统
class RetrievalSystem:
    def search(self, query):
        # 这里应该是检索相关信息的逻辑
        # 返回一个假设的文档列表
        return [{"content": "This is a retrieved document related to the query."}]

# 假设的语言模型
class LanguageModel:
    def generate(self, messages, context):
        # 这里应该是基于消息和上下文生成回应的逻辑
        # 返回一个假设的生成回应
        return "This is a generated response based on the input messages and context."

# 假设的 RAG SDK
class RAGSDK:
    def __init__(self, retrieval_system, language_model):
        self.retrieval_system = retrieval_system
        self.language_model = language_model

    def generate_response(self, model_name, messages, temperature=0.5, max_tokens=1000):
        # 假设逻辑来处理消息，检索信息，并生成回应
        query = messages[-1]['content']  # 假设最后一条消息是用户查询
        context = self.retrieval_system.search(query)
        response = self.language_model.generate(messages, context)
        return response

# 以下是使用 RAGSDK 的示例代码
def rag_sdk(model_name, messages, temperature=0.5, max_tokens=1000):
    try:
        # 初始化检索系统和语言模型
        retrieval_system = RetrievalSystem()
        language_model = LanguageModel()
        rag_sdk_instance = RAGSDK(retrieval_system, language_model)

        # 生成响应
        response = rag_sdk_instance.generate_response(model_name, messages, temperature, max_tokens)
        logging.info("Response generated successfully.")
        return response
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None

# 示例用法
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)

    # 消息列表
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is LDA?"}
    ]

    # 调用函数
    reply = rag_sdk("rag-model", messages)
    if reply:
        print("Generated response:", reply)
    else:
        print("Failed to generate a response.")


INFO:root:Response generated successfully.


Generated response: This is a generated response based on the input messages and context.


## 转向量

In [71]:
import openai
import numpy as np
import pandas as pd 
from sklearn.neighbors import NearestNeighbors
from transformers import AutoModel, AutoTokenizer
import torch

# 初始化模型和分词器
model_name = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state
    cls_embedding = last_hidden_state[:, 0, :]
    return cls_embedding.detach().numpy()

# 读取CSV文件中的问题和答案
csv_file_path = '问题集.csv'  # 替换为你的CSV文件路径
df = pd.read_csv(csv_file_path)  # 默认情况下，pandas会使用第一行作为列标题
questions = df['问题内容'].tolist()  # 标题是"问题内容"
answers = df['答案内容'].tolist()  # 标题是"答案内容"

# 获取每个问题的嵌入向量
embeddings = [get_embedding(doc) for doc in questions]

# 将嵌入向量转换为NumPy数组，以便进行相似性搜索
embeddings_matrix = np.vstack(embeddings)

# 使用sklearn的NearestNeighbors构建一个索引
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(embeddings_matrix)

# 查询功能，返回最相似的问题索引和答案
def query(text, k=len(questions)):
    query_embedding = get_embedding(text).reshape(1, -1)
    distances, indices = nbrs.kneighbors(query_embedding, n_neighbors=k)
    # 返回所有相似问题及其答案和距离
    return [(index, answers[index], distance) for index, distance in zip(indices[0], distances[0])]

# 用一个查询来测试
query_text = "可视化"
similar_questions = query(query_text)
for rank, (index, answer, distance) in enumerate(similar_questions, start=1):
    print(f"Rank {rank}:")
    print(f"Most similar question: {questions[index]}")
    print(f"Answer: {answer}")
    print(f"Distance: {distance}")
    print()  # 空行分隔各个结果

Rank 1:
Most similar question: 什么是数据可视化？
Answer: 数据可视化是将数据转换为图形或图表形式的过程，以便更容易分析和理解。
Distance: 2.17663448362023

Rank 2:
Most similar question: 如何使用卷积神经网络(CNN)处理图像？
Answer: CNN是一种特殊的神经网络，通过卷积层提取图像特征，用于图像处理任务。
Distance: 2.968803898730111

Rank 3:
Most similar question: 什么是过拟合？
Answer: 过拟合是指模型在训练数据上学习得太好，导致泛化能力差。
Distance: 3.002709095996682

Rank 4:
Most similar question: 什么是机器学习？
Answer: 机器学习是使计算机能够从数据中学习并做出预测或决策的学科领域，而无需每一步都进行明确编程。
Distance: 3.031502297215975

Rank 5:
Most similar question: 如何在pandas中处理缺失数据？
Answer: 可以使用dropna()删除或fillna()填充缺失数据。
Distance: 3.052775521979715

Rank 6:
Most similar question: 什么是Transformer模型？
Answer: Transformer模型是一种基于自注意力机制的深度学习模型，常用于NLP任务。
Distance: 3.1214053680562994

Rank 7:
Most similar question: 如何在Sklearn中实现KNN算法？
Answer: 可以使用sklearn.neighbors中的KNeighborsClassifier类来实现KNN（k最近邻）算法。
Distance: 3.132223478206783

Rank 8:
Most similar question: 什么自然语言处理(NLP)？
Answer: 自然语言处理是计算机科学、人工智能和语言学领域的一个分支，用于处理人类语言。
Distance: 3.1586851536071765

Rank 9:
Most simila

In [58]:
def combined_query_and_reply(query_text):
    # 使用向量匹配找到最相似的文档
    indices, distances = query(query_text)
    most_similar_doc_index = indices[0][0]  # 获取最相似文档的索引
    most_similar_doc = documents[most_similar_doc_index]  # 获取最相似的文档内容

    # 准备要发送给大模型的信息
    messages = [
        {"role": "system", "content": "你是一个帮助者。"},
        {"role": "user", "content": f"我想问一下与以下内容相关的信息：{query_text}"},
        {"role": "assistant", "content": f"根据您的查询，最相似的文档是：{most_similar_doc}。以下是关于该文档的详细信息："}
    ]

    # 调用 OpenAI SDK 来获取回复
    response = open_ai_sdk("gpt-4-1106-preview", messages)
    if response:
        try:
            # 检查 Choice 对象有哪些属性
            choice = response.choices[0]
            assistant_reply = choice.message  # 假设 message 是正确的属性

            # 打印最相似的文档和聊天机器人的回复
            print(f"最相似的文档：{most_similar_doc}")
            # print("聊天机器人的回复：", assistant_reply)  # 也先注释掉
        except (AttributeError, IndexError) as e:
            # 如果出现属性错误或索引错误，记录日志
            logger.error(f"Error accessing the response data: {e}")
    else:
        print("未能从 OpenAI API 获取有效响应。")

# 测试组合查询和回复
if __name__ == '__main__':
    query_text = "我想找有关文档内容的信息"
    combined_query_and_reply(query_text)

INFO:httpx:HTTP Request: POST http://gpt-proxy.jd.com/gateway/azure/chat/completions "HTTP/1.1 200 "
INFO:__main__:API response received.


最相似的文档：文档一的内容
